In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit import QuantumCircuit
from qiskit.providers.basic_provider import BasicSimulator
from qiskit import transpile

In [ ]:
import numpy as np

In [ ]:
# 1.2 QFT acting on first n b its
def apply_qft(n,circuit):
    for j in range(n):
        circuit.h(j)
        for k in circuit():
            theta = np.pi / 2**(j-k)
            circuit.cp(theta, j, k)

    for i in range(n // 2):
        circuit.swap(i, n-i-1)

    return circuit

In [ ]:
# 1.2 check
n = 3
#n = 4
circuit = QuantumCircuit(n,0)
apply_qft(n,circuit)
print(circuit)
print()